In [15]:
import csv
import xml.etree.ElementTree as ET

In [16]:
class Block:
    
    def __init__(self, name, x, y, typ, node):
        self.name = name
        self.x = x
        self.y = y
        self.type = typ
        self.node = node

In [17]:
def get_place_data(testbench_name):
    
    testbench = testbench_name + ".v/common"
    
    f2 = open(testbench + "/" + testbench_name + ".place", "r")
    
    b = f2.readlines()[5:]
    
    place_matrix = []
    for i in b:
        add = i.strip().split('\t')
#         print(add)
        if '' in add:
            del add[1]
        place_matrix.append(add)
    
    place_dict = {idx[0] : idx[1:] for idx in place_matrix}
    
    return place_dict

In [18]:
def get_block_list(testbench_name, place_dict):
    
    testbench = testbench_name + ".v/common"

    tree = ET.parse(testbench + "/" + testbench_name + ".net")
    root = tree.getroot()
    
    block_list = []
    for child in root:
        if 'name' in child.attrib.keys():
            name = child.attrib['name']
            if 'io' in child.attrib['instance']:
                typ = "io"
            elif 'mult_36' in child.attrib['instance']:
                typ = "complex"
            elif 'memory' in child.attrib['instance']:
                typ = "complex"
            else:
                typ = "none"
            x = int(place_dict[child.attrib['name']][0])
            y = int(place_dict[child.attrib['name']][1])
            b = Block(name, x, y, typ, child)
            block_list.append(b)
            
    return block_list

In [126]:
def get_critical_paths(testbench_name, block_list):
    testbench = testbench_name + ".v/common"
    
    f1 = open(testbench + "/report_timing.setup.rpt", "r")
    a = f1.readlines()[4:]
    
    

    crit_path_info = []
    path_num = 0
    line_num = 0
    while(path_num < 1):
        path_num += 1
        m = []
        while(True):
            if(a[line_num] == "#Path " + str(path_num+1) +"\n"):
                break
            else:
                m.append(a[line_num])
                line_num += 1
        crit_path_info.append(m)

#     print(crit_path_info)
    slacks =[]
    k = 0    
    for path_no in range(1):
        
        while(True):
#             print(k)
            i = a[k]
            i = i.strip().split()
            k += 1
            if "slack" in i:
#                 print(i)
                slacks.append(i[2])
                break
            
        
        
    path_list = []
    for path in crit_path_info:
        k = []
        for j in path[10:]:
# #         print(j)
            if(j == "\n"):
                break
            else:
                k.append(j)
        path_list.append(k)
            
    
#     print(path_list)
    path_matrix = []
    for i in range(len(path_list)):
        temp = []
        for j in range(len(path_list[i])):
#         print(path[i][j])
            temp.append(path_list[i][j].strip().split())
        path_matrix.append(temp)

#     print(path_matrix)
    path_ll = []
    time_ll = []
    for p in path_matrix:
        k = p[:-1]
        k1 = [item[0] for item in k]
        k2 = [item[-2] for item in k]
        time_ll.append(k2)
        path_ll.append(k1)
    
#     print(path_ll)
    
# # #     path_ll = []
# # path_ll = [item[0] for item in path_matrix]

    path_nodes = []
    for p in path_ll:
        k = []
        for n in p:
            temp = n.split('.')
            temp.pop()
            temp = '.'.join(temp)
            k.append(temp)
        
        path_nodes.append(k)
    
    
#     print(len(path_nodes)==len(time_ll))
    path_nodes_set = []
#     for path in path_nodes:
#         nodes_set = []
#         for node in path:
# #         print(node)
#             if node not in nodes_set:
# #             print(node)
#                 nodes_set.append(node)
# #     print(nodes_set)
#         path_nodes_set.append(nodes_set)

    print(time_ll)
    time_set = []
    for i in range(len(path_nodes)):
        nodes_set = []
        for j in range(len(path_nodes[i])):
            node = path_nodes[i][j]
            if node not in nodes_set:
                n_t = [node, time_ll[i][j]]
                nodes_set.append(n_t)
#                 time_set.append(time_ll[i][j])
#                 print(node, time_ll[i][j], j)
        path_nodes_set.append(nodes_set)
    
# pd = get_place_data("or1200")
    
    
# block_list = get_block_list("or1200", pd)
    
    path_list = []
    for nodes_set in path_nodes_set:
#     print("hello")
        temp = []
        for nodes in nodes_set:
#         print(nodes_set)
#         for node in nodes_set:
#             print(node)
                for block in block_list:
                    for b in block.node.iter('block'):
                        if(b.attrib['name'] == nodes[0]):
                            temp.append([block, nodes[1]])
                            break
        path_list.append(temp)
        
        
    return path_list, slacks

In [127]:
tb_list = [
#     "arm_core",
#     "bgm",
#     "blob_merge",
#     "boundtop",
#     "ch_intrinsics",
    "diffeq1",
#     "diffeq2",
#     "LU8PEEng",
#     "LU32PEEng",
#     "mcml",
#     "mkDelayWorker32B",
#     "mkPktMerge",
#     "mkSMAdapter4B",
#     "or1200",
#     "raygentop",
#     "sha",
#     "spree",
#     "stereovision0",
#     "stereovision1",
#     "stereovision2",
#     "stereovision3",
]

In [128]:
tb = "diffeq1"

In [129]:
pl = get_place_data(tb)
bl = get_block_list(tb, pl)
cl = get_critical_paths(tb, bl)

[['0.042', '0.124', '0.794', '1.523', '0.514', '1.523', '0.513', '1.523', '0.596', '0.195', '0.000', '0.300', '0.000', '0.010', '0.000', '0.010', '0.000', '0.010', '0.000', '0.010', '0.000', '0.300', '0.675', '0.300', '0.000', '0.010', '0.000', '0.010', '0.000', '0.010', '0.000', '0.010', '0.000', '0.010', '0.000', '0.010', '0.160', '0.010', '0.000', '0.010', '0.000', '0.010', '0.000', '0.010', '0.000', '0.010', '0.000', '0.010', '0.000', '0.010', '0.000', '0.010', '0.000', '0.010', '0.000', '0.300', '0.479', '0.261', '0.000']]


In [153]:
def routing_delay(cl):
    
    delay = 0
    cl = cl[0][0]
    print(len(cl))
    for i in range(len(cl) - 1):
#         print(i)
        if(cl[i][0] != cl[i+1][0]):
            delay += float(cl[i+1][1])
            print(cl[i+1][0].name)
    return delay

In [154]:
d = routing_delay(cl)

59
diffeq_paj_convert^MUL~0[0]
diffeq_paj_convert^MUL~42[0]
diffeq_paj_convert^MUL~43-0[0]
diffeq_paj_convert^MIN~44-0[0]
diffeq_paj_convert^MIN~47-0[0]
diffeq_paj_convert^MIN~47-20[0]
diffeq_paj_convert^MIN~44-20[0]


In [155]:
d

3.7310000000000008